In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nlppln

wf = nlppln.WorkflowGenerator()
wf.load(steps_dir='../cwl/')
print(wf.list_steps())

in_dir = wf.add_inputs(in_dir='Directory')
ocr_char_counts = wf.add_inputs(ocr_n='string?')
gs_char_counts = wf.add_inputs(gs_n='string?')
align_metadata = wf.add_inputs(align_m='string?')
align_changes = wf.add_inputs(align_c='string?')

#folia_files = wf.ls(in_dir=in_dir)
folia_files = wf.select_folia_files(in_dir=in_dir)
gs_with_empty, ocr_with_empty = wf.folia2ocr_and_gs(in_file=folia_files, scatter='in_file', scatter_method='flat_crossproduct')
gs, ocr = wf.remove_empty_files(gs_files=gs_with_empty, ocr_files=ocr_with_empty)

ocr_cc = wf.count_chars(in_file=ocr, scatter='in_file', scatter_method='dotproduct')
ocr_char_counts = wf.merge_json(in_files=ocr_cc, name=ocr_char_counts)

gs_cc = wf.count_chars(in_file=gs, scatter='in_file', scatter_method='dotproduct')
gs_char_counts = wf.merge_json(in_files=gs_cc, name=gs_char_counts)

changes_files, metadata_files = wf.align(file1=ocr, file2=gs, scatter=['file1', 'file2'], scatter_method='dotproduct')
metadata = wf.merge_json(in_files=metadata_files, name=align_metadata)
changes = wf.merge_json(in_files=changes_files, name=align_changes)

wf.add_outputs(gs=gs, ocr=ocr, ocr_char_counts=ocr_char_counts, gs_char_counts=gs_char_counts, metadata=metadata, changes=changes)
wf.save('../cwl/folia2texts.cwl')